# Day 22: 画像認識プロジェクト - データセット準備と前処理

## Learning Objectives
- 手書き数字データを生成する
- 画像前処理パイプラインを実装する
- 特徴抽出技術を理解する
- データ拡張の手法を学ぶ

---

# Part 1: Theory (2 hours)

## 1.1 画像認識の概要

画像認識（Image Recognition）とは、画像から特定の物体やパターンを識別する技術です。

**手書き数字認識の意義**:
- 最も基本的な画像認識タスク
- 機械学習の分野における「Hello, World」
- OCR（光学文字認識）の基礎
- 他の複雑な認識タスクの土台

### MNISTデータセット

MNIST（Modified National Institute of Standards and Technology）は、機械学習で最も有名なデータセットの一つです。

**特徴**:
- 70,000枚の手書き数字画像（60,000枚学習用 + 10,000枚テスト用）
- 各画像は28×28ピクセル（グレースケール）
- ラベル：0-9の数字（教師あり学習）

**画像構造**:
画像サイズ: 28×28 = 784ピクセル
画素値: 0(黒) - 255(白)
チャンネル数: 1（グレースケール）
    
例: 数字 '5' の28×28ピクセル配列
```

## 1.2 データの生成と生成モデル

### ランダムな手書き数字の生成

実際のデータセットを使用せず、プログラムで手書き数字を生成する方法です。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
from sklearn.datasets import make_classification
import random

# matplotlibで日本語対応
plt.rcParams['font.family'] = 'Meiryo'  # Windowsの場合
plt.rcParams['axes.unicode_minus'] = False  # 負号の表示

### 簡単な手書き数字の生成（バージョン1）

In [ ]:
def create_simple_digit(digit, size=28, width=20):
    """シンプルな数字を生成する関数"""
    # 白い背景の画像を作成
    img = np.ones((size, size), dtype=np.uint8) * 255
    
    # 中心位置
    center = size // 2
    
    if digit == 0:  # 円
        cv2.circle(img, (center, center), width//2, 0, thickness=3)
    elif digit == 1:  # 縦棒
        cv2.line(img, (center-2, center-width//2), (center-2, center+width//2), 0, 3)
    elif digit == 2:  # 2
        cv2.line(img, (center-width//2, center-width//2), (center+width//2, center-width//2), 0, 3)  # 上横
        cv2.line(img, (center+width//2, center-width//2), (center+width//2, center), 0, 3)  # 右縦
        cv2.line(img, (center+width//2, center), (center-width//2, center), 0, 3)  # 中横
        cv2.line(img, (center-width//2, center), (center-width//2, center+width//2), 0, 3)  # 左縦
        cv2.line(img, (center-width//2, center+width//2), (center+width//2, center+width//2), 0, 3)  # 下横
    
    return img

# 0-9の数字を生成して表示
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
axes = axes.flatten()

for i, ax in enumerate(axes):
    digit_img = create_simple_digit(i)
    ax.imshow(digit_img, cmap='gray')
    ax.set_title(f'Digit: {i}')
    ax.axis('off')

plt.tight_layout()
plt.show()

### より自然な手書き数字の生成（バージョン2）

In [ ]:
def create_handwritten_digit(digit, size=28):
    """より自然な手書き数字を生成する関数"""
    # 白い背景
    img = Image.new('L', (size, size), 255)
    draw = ImageDraw.Draw(img)
    
    # ランダムな傾きとノイズを追加
    angle = random.randint(-10, 10)
    offset_x = random.randint(-2, 2)
    offset_y = random.randint(-2, 2)
    
    # パラメトリックな数式で数字を描画
    center_x, center_y = size//2 + offset_x, size//2 + offset_y
    
    if digit == 0:  # 0 - 楕円形
        # 外円
        draw.ellipse([center_x-10, center_y-12, center_x+10, center_y+12], outline=0, width=3)
    elif digit == 1:  # 1 - 斜めの直線
        # 縦棒（少し右に傾く）
        points = [(center_x-2, center_y-10), (center_x+1, center_y+10)]
        draw.line(points, fill=0, width=3)
        # 上横棒
        points = [(center_x-2, center_y-10), (center_x+2, center_y-10)]
        draw.line(points, fill=0, width=2)
    elif digit == 2:  # 2
        # 上弧
        draw.arc([center_x-10, center_y-12, center_x+10, center_y-2], 0, 180, fill=0, width=3)
        # 横棒
        draw.line([center_x+10, center_y-2, center_x-10, center_y+2], fill=0, width=3)
        # 下弧
        draw.arc([center_x-10, center_y+2, center_x+10, center_y+12], 180, 360, fill=0, width=3)
    
    # ノイズを追加
    for _ in range(20):
        x = random.randint(2, size-3)
        y = random.randint(2, size-3)
        draw.point([(x, y)], fill=random.randint(0, 100))
    
    return np.array(img)

# 時間がかかるので、5つの数字だけを生成
fig, axes = plt.subplots(1, 5, figsize=(10, 2))

for i, ax in enumerate(axes):
    digit_img = create_handwritten_digit(i * 2)  # 0, 2, 4, 6, 8
    ax.imshow(digit_img, cmap='gray')
    ax.set_title(f'Digit: {i*2}')
    ax.axis('off')

plt.tight_layout()
plt.show()

## 1.3 画像前処理パイプライン

画像認識の精度を向上させるため、画像に対して一連の前処理を行います。

In [ ]:
def preprocess_image(image):
    """画像前処理パイプライン
    
    前処理ステップ:
    1. グレースケール化（すでであれば不要）
    2. サイズ統一（28×28）
    3. 二値化（必要に応じて）
    4. 正規化（0-1）
    5. ゼロパディング（必要に応じて）
    """
    # 2. サイズ統一
    if image.shape != (28, 28):
        image = cv2.resize(image, (28, 28), interpolation=cv2.INTER_AREA)
    
    # 3. 二値化（閾値処理）
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # 4. 正規化（0-1）
    normalized = binary / 255.0
    
    # 5. 中央値フィルタ（ノイズ除去）
    denoised = cv2.medianBlur((normalized * 255).astype(np.uint8), 3)
    denoised = denoised / 255.0
    
    return denoised

# 前処理の実例
fig, axes = plt.subplots(2, 3, figsize=(10, 7))
axes = axes.flatten()

# 元の画像
original = create_handwritten_digit(7)
axes[0].imshow(original, cmap='gray')
axes[0].set_title('Original (255)')
axes[0].axis('off')

# 二値化
_, binary = cv2.threshold(original, 127, 255, cv2.THRESH_BINARY)
axes[1].imshow(binary, cmap='gray')
axes[1].set_title('Binary')
axes[1].axis('off')

# 正規化
normalized = binary / 255.0
axes[2].imshow(normalized, cmap='gray')
axes[2].set_title('Normalized (0-1)')
axes[2].axis('off')

# ノイズ除去
denoised = cv2.medianBlur(binary, 3)
axes[3].imshow(denoised, cmap='gray')
axes[3].set_title('Denoised')
axes[3].axis('off')

# 閾値調整
_, binary_thresh = cv2.threshold(original, 150, 255, cv2.THRESH_BINARY)
axes[4].imshow(binary_thresh, cmap='gray')
axes[4].set.title('Binary (thresh=150)')
axes[4].axis('off')

# 最終前処理
final_processed = preprocess_image(original)
axes[5].imshow(final_processed, cmap='gray')
axes[5].set_title('Final Processed')
axes[5].axis('off')

plt.tight_layout()
plt.show()

## 1.4 特徴抽出の基礎

特徴抽出とは、画像から認識に有用な情報を抽出するプロセスです。

### 特徴量の種類

#### 1. ピクセルレベルの特徴
- ピクセル値そのもの
- ヒストグラム（明度分布）

#### 2. 統計的特徴
- 平均、分散、標準偏差
- エントロピー（ランダムさ）

#### 3. 形状的特徴
- 重心
- 横長・縦長の比率
- モーメント（慣性モーメント）

#### 4. エッジ特徴
- エッジの数
- エッジの方向分布

#### 5. トポロジカル特徴
- 穴の数
- 結合成分の数

In [ ]:
def extract_features(image):
    """画像から特徴を抽出する関数"""
    features = {}
    
    # 1. 基本的な統計量
    features['mean'] = np.mean(image)
    features['std'] = np.std(image)
    features['min'] = np.min(image)
    features['max'] = np.max(image)
    
    # 2. 画像のサイズ情報
    features['height'] = image.shape[0]
    features['width'] = image.shape[1]
    features['aspect_ratio'] = features['width'] / features['height']
    
    # 3. ピクセル値のヒストグラム
    hist, _ = np.histogram(image.flatten(), bins=10, range=(0, 1))
    features['histogram'] = hist
    features['histogram_peak'] = np.argmax(hist) / 10  # ピーク位置
    
    # 4. 重心（ピクセルの質量中心）
    y, x = np.mgrid[0:image.shape[0], 0:image.shape[1]]
    # 濃度が濃いほど重みが大きくなる
    weighted_sum = np.sum(image)
    if weighted_sum > 0:
        features['centroid_x'] = np.sum(x * image) / weighted_sum
        features['centroid_y'] = np.sum(y * image) / weighted_sum
    else:
        features['centroid_x'] = image.shape[1] / 2
        features['centroid_y'] = image.shape[0] / 2
    
    # 5. 分散（画像の広がり）
    if weighted_sum > 0:
        features['variance_x'] = np.sum((x - features['centroid_x'])**2 * image) / weighted_sum
        features['variance_y'] = np.sum((y - features['centroid_y'])**2 * image) / weighted_sum
    else:
        features['variance_x'] = 0
        features['variance_y'] = 0
    
    # 6. エッジ検出
    # Sobelフィルタ
    sobel_x = cv2.Sobel(image.astype(np.float32), cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image.astype(np.float32), cv2.CV_64F, 0, 1, ksize=3)
    
    features['edge_magnitude'] = np.sqrt(sobel_x**2 + sobel_y**2).mean()
    features['edge_direction'] = np.arctan2(sobel_y.mean(), sobel_x.mean())
    
    return features

# 特徴抽出の実例
sample_digit = create_handwritten_digit(4)
processed_digit = preprocess_image(sample_digit)
features = extract_features(processed_digit)

print("抽出された特徴:")
for key, value in features.items():
    if key != 'histogram':
        print(f"{key}: {value:.4f}")

# ヒストグラムの表示
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(processed_digit, cmap='gray')
plt.title('Processed Digit')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.bar(range(10), features['histogram'])
plt.xlabel('Bin (0-1)')
plt.ylabel('Count')
plt.title('Pixel Value Histogram')
plt.show()

## 1.5 データ拡張（Data Augmentation）

データ拡張は、既存のデータから新しいデータを作成してデータセットを増やす技術です。

In [ ]:
def augment_image(image, rotation_range=10, noise_level=0.1):
    """データ拡張関数
    
    Args:
        image: 入力画像
        rotation_range: 回転角度の範囲（度）
        noise_level: ノイズの強さ（0-1）
    """
    augmented = image.copy()
    
    # 1. 回転
    angle = np.random.uniform(-rotation_range, rotation_range)
    rows, cols = image.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    augmented = cv2.warpAffine(augmented, M, (cols, rows), flags=cv2.INTER_CUBIC)
    
    # 2. ノイズ追加
    noise = np.random.normal(0, noise_level, image.shape)
    augmented = np.clip(augmented + noise, 0, 1)
    
    # 3. スケーリング（拡大縮小）
    scale = np.random.uniform(0.9, 1.1)
    scaled = cv2.resize(augmented, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    
    # トリミングして元のサイズに戻す
    if scale > 1:
        # 拡大の場合は中心をトリミング
        h, w = scaled.shape
        start_h = (h - rows) // 2
        start_w = (w - cols) // 2
        augmented = scaled[start_h:start_h+rows, start_w:start_w+cols]
    elif scale < 1:
        # 縮小の場合はパディング
        h, w = scaled.shape
        pad_h = (rows - h) // 2
        pad_w = (cols - w) // 2
        augmented = np.pad(scaled, ((pad_h, rows - h - pad_h), (pad_w, cols - w - pad_w)), 'constant')
    
    # 4. 明るさ調整
    brightness_factor = np.random.uniform(0.8, 1.2)
    augmented = np.clip(augmented * brightness_factor, 0, 1)
    
    return augmented

# データ拡張の例
base_digit = create_handwritten_digit(8)
processed_base = preprocess_image(base_digit)

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

# 元の画像
axes[0].imshow(processed_base, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

# 拡張例
for i in range(1, 8):
    augmented = augment_image(processed_base)
    axes[i].imshow(augmented, cmap='gray')
    axes[i].set_title(f'Augmented {i}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

### さまざまな拡張手法の比較

In [ ]:
def create_augmented_samples(original, num_samples=16):
    """拡張サンプルを作成する"""
    samples = []
    
    # 元の画像
    samples.append(('Original', original))
    
    # 種々の拡張パターン
    for i in range(num_samples - 1):
        # 異なるパラメータで拡張
        rotation = 15 if i < 5 else 0  # 最初の5つだけ回転
        noise = 0.05 + (i % 3) * 0.05  # ノイズレベルを変化
        
        augmented = augment_image(original, rotation_range=rotation, noise_level=noise)
        
        # パターン名
        if rotation > 0 and noise > 0.1:
            name = f'Rot+Noise{i-4}'
        elif rotation > 0:
            name = f'Rot{i+1}'
        else:
            name = f'Noise{i-4}'
            
        samples.append((name, augmented))
    
    return samples

# 拡張サンプルのグリッド表示
def display_augmentation_grid(samples, title="Data Augmentation Examples"):
    """拡張サンプルをグリッドで表示"""
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, (name, img) in enumerate(samples):
        if i < len(axes):
            axes[i].imshow(img, cmap='gray')
            axes[i].set_title(name)
            axes[i].axis('off')
    
    # 空白のサブプロットを非表示
    for i in range(len(samples), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# 拡張サンプルを作成して表示
augmented_samples = create_augmented_samples(processed_base, 16)
display_augmentation_grid(augmented_samples)

## 1.6 データセットの分割

学習データとテストデータを分割する重要性と方法です。

In [ ]:
def create_dataset(num_samples_per_digit=500, test_ratio=0.2, augment=True):
    """手書き数字データセットを作成する"""
    dataset = {
        'X_train': [], 'y_train': [],
        'X_test': [], 'y_test': [],
        'X_raw': [], 'y_raw': []
    }
    
    # それぞれの数字についてデータを作成
    for digit in range(10):
        print(f"Digit {digit} の生成中...")
        
        # 基本サンプル
        raw_samples = []
        for _ in range(num_samples_per_digit):
            raw_sample = create_handwritten_digit(digit)
            raw_samples.append(raw_sample)
        
        # 拡張
        if augment:
            augmented_samples = []
            for sample in raw_samples:
                # 2つの拡張サンプルを追加
                for _ in range(2):
                    augmented = augment_image(preprocess_image(sample))
                    augmented_samples.append(augmented)
            
            # 元のサンプルも追加
            all_samples = [preprocess_image(sample) for sample in raw_samples] + augmented_samples
        else:
            all_samples = [preprocess_image(sample) for sample in raw_samples]
        
        # データの分割
        n_samples = len(all_samples)
        n_test = int(n_samples * test_ratio)
        
        # ランダムにテストデータを選択
        indices = np.random.permutation(n_samples)
        test_indices = indices[:n_test]
        train_indices = indices[n_test:]
        
        # データセットに追加
        dataset['X_train'].extend([all_samples[i] for i in train_indices])
        dataset['y_train'].extend([digit] * len(train_indices))
        dataset['X_test'].extend([all_samples[i] for i in test_indices])
        dataset['y_test'].extend([digit] * len(test_indices))
        dataset['X_raw'].extend(raw_samples)
        dataset['y_raw'].extend([digit] * len(raw_samples))
    
    # NumPy配列に変換
    for key in dataset:
        if key.startswith('X_'):
            dataset[key] = np.array(dataset[key])
        elif key.startswith('y_'):
            dataset[key] = np.array(dataset[key])
    
    return dataset

# データセットの作成（時間がかかるので少なめに）
print("データセットの生成を開始...")
dataset = create_dataset(num_samples_per_digit=100, test_ratio=0.2, augment=True)

# データセットの情報表示
print(f"\nデータセットのサイズ:")
print(f"学習データ: {dataset['X_train'].shape} (画像), {dataset['y_train'].shape} (ラベル)")
print(f"テストデータ: {dataset['X_test'].shape} (画像), {dataset['y_test'].shape} (ラベル)")

# データセットの分布を確認
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
unique, counts = np.unique(dataset['y_train'], return_counts=True)
plt.bar(unique, counts)
plt.title('Training Data Distribution')
plt.xlabel('Digit')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
unique, counts = np.unique(dataset['y_test'], return_counts=True)
plt.bar(unique, counts)
plt.title('Test Data Distribution')
plt.xlabel('Digit')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

## 1.7 特徴の可視化

抽出した特徴を可視化し、異なる数字の特徴の違いを確認します。

In [ ]:
def extract_features_for_dataset(dataset):
    """データセット全体から特徴を抽出する"""
    features_per_digit = {i: [] for i in range(10)}
    
    # 最初の100サンプルから特徴を抽出（時間節約）
    max_samples = 100
    
    for digit in range(10):
        # この数字のデータからサンプリング
        digit_indices = np.where(dataset['y_train'] == digit)[0]
        sample_indices = np.random.choice(digit_indices, min(max_samples, len(digit_indices)), replace=False)
        
        for idx in sample_indices:
            features = extract_features(dataset['X_train'][idx])
            features_per_digit[digit].append(features)
    
    return features_per_digit

# 特徴抽出（時間がかかるので注意）
print("特徴抽出を開始...")
features_per_digit = extract_features_for_dataset(dataset)

# 特徴の平均値を計算
feature_means = {i: {} for i in range(10)}
for digit in range(10):
    for key in features_per_digit[digit][0].keys():
        if key != 'histogram':
            values = [f[key] for f in features_per_digit[digit]]
            feature_means[digit][key] = np.mean(values)

# 特徴を可視化
plt.figure(figsize=(15, 10))

# 1. 平均値の比較
plt.subplot(2, 2, 1)
means = [feature_means[d]['mean'] for d in range(10)]
plt.bar(range(10), means)
plt.title('Mean Pixel Value')
plt.xlabel('Digit')
plt.ylabel('Mean')

# 2. 分散の比較
plt.subplot(2, 2, 2)
vars_ = [feature_means[d]['std']**2 for d in range(10)]  # 分散 = 標準偏差^2
plt.bar(range(10), vars_)
plt.title('Variance')
plt.xlabel('Digit')
plt.ylabel('Variance')

# 3. エッジ強度の比較
plt.subplot(2, 2, 3)
edges = [feature_means[d]['edge_magnitude'] for d in range(10)]
plt.bar(range(10), edges)
plt.title('Edge Magnitude')
plt.xlabel('Digit')
plt.ylabel('Edge Strength')

# 4. 重心のX座標比較
plt.subplot(2, 2, 4)
centroids_x = [feature_means[d]['centroid_x'] for d in range(10)]
plt.bar(range(10), centroids_x)
plt.title('Centroid X Position')
plt.xlabel('Digit')
plt.ylabel('X Position')

plt.tight_layout()
plt.show()

---

# Part 2: Practice (2 hours)

それでは、学んだ知識を実際に使ってみましょう！

## Exercise 2.1: データセットの拡充

手書き数字データセットを1000サンプル/数字作成し、CSVファイルに保存せよ。

In [ ]:
def save_large_dataset(num_samples=1000, filename='handwritten_digits.csv'):
    """大きいデータセットを作成して保存する"""
    import pandas as pd
    
    print(f"{num_samples}サンプル/数字のデータセット作成中...")
    
    # データを保存するリスト
    data = []
    
    for digit in range(10):
        print(f"Digit {digit} を生成中...")
        
        for i in range(num_samples):
            # 画像生成
            raw_img = create_handwritten_digit(digit)
            processed_img = preprocess_image(raw_img)
            
            # 画像を1次元に変換
            flattened = processed_img.flatten()
            
            # ラベルを追加
            row = [digit] + flattened.tolist()
            
            data.append(row)
    
    # DataFrameに変換
    columns = ['label'] + [f'pixel_{i}' for i in range(28*28)]
    df = pd.DataFrame(data, columns=columns)
    
    # CSVとして保存
    df.to_csv(filename, index=False)
    print(f"データセットを {filename} に保存しました")
    
    return df

# 小規模版で実行
df_small = save_large_dataset(num_samples=100, filename='small_digits.csv')
print(f"\nデータセットの形状: {df_small.shape}")
print(f"最初の5行:\n{df_small.head()}")

## Exercise 2.2: 前処理の最適化

異なる前処理方法を試し、最適な組み合わせを見つけよ。

In [ ]:
def preprocess_with_methods(image, methods):
    """指定された前処理メソッドを適用する"""
    processed = image.copy().astype(np.float64)
    
    for method in methods:
        if method == 'normalize':
            # 正規化
            processed = (processed - processed.min()) / (processed.max() - processed.min())
        elif method == 'binarize':
            # 二値化（適切な閾値で）
            threshold = np.percentile(processed, 50)
            processed = (processed > threshold).astype(float)
        elif method == 'equalize':
            # ヒストグラム平坦化
            processed = cv2.equalizeHist((processed * 255).astype(np.uint8)) / 255.0
        elif method == 'denoise':
            # ノイズ除去
            processed = cv2.medianBlur((processed * 255).astype(np.uint8), 3) / 255.0
        elif method == 'blur':
            # ぼかし（ノイズ除去の代わり）
            processed = cv2.GaussianBlur(processed, (3, 3), 0)
        elif method == 'threshold':
            # 固定閾値
            _, processed = cv2.threshold((processed * 255).astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
            processed = processed / 255.0
    
    return processed

# 異なる前処理パイプラインの比較
def compare_preprocessing_methods(test_digit=3):
    """前処理方法の比較"""
    # テスト用画像
    original = create_handwritten_digit(test_digit)
    
    # 前処理パイプラインの定義
    pipelines = {
        'Original': original,
        'Normalize Only': preprocess_with_methods(original, ['normalize']),
        'Binary Only': preprocess_with_methods(original, ['binarize']),
        'Denoised': preprocess_with_methods(original, ['normalize', 'denoise']),
        'Equalized': preprocess_with_methods(original, ['normalize', 'equalize']),
        'Threshold': preprocess_with_methods(original, ['normalize', 'threshold']),
        'Full Pipeline': preprocess_with_methods(original, ['normalize', 'denoise', 'threshold'])
    }
    
    # 表示
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.flatten()
    
    for i, (name, img) in enumerate(pipelines.items()):
        if i < len(axes):
            axes[i].imshow(img, cmap='gray')
            axes[i].set_title(name)
            axes[i].axis('off')
    
    plt.suptitle(f'Digit {test_digit} - Preprocessing Comparison', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # 特徴比較
    features_comparison = {}
    for name, img in pipelines.items():
        if name != 'Original':
            features_comparison[name] = extract_features(img)
    
    return features_comparison

# 前処理方法の比較
features_comp = compare_preprocessing_methods(7)

# 特徴の比較
print("\n特徴量の比較:")
for method, features in features_comp.items():
    print(f"\n{method}:")
    print(f"  平均: {features['mean']:.4f}")
    print(f"  標準偏差: {features['std']:.4f}")
    print(f"  エッジ強度: {features['edge_magnitude']:.4f}")

## Exercise 2.3: カスタム特徴量の実装

新たな特徴量を3つ以上追加し、その有用性を評価せよ。

In [ ]:
def extract_advanced_features(image):
    """高度な特徴量を抽出する関数"""
    features = {}
    
    # 画像サイズ
    h, w = image.shape
    
    # 1. 水平/垂直方向の投影プロファイル
    horizontal_proj = np.sum(image, axis=1)  # 行方向の合計
    vertical_proj = np.sum(image, axis=0)    # 列方向の合計
    
    features['horizontal_max'] = np.max(horizontal_proj)
    features['horizontal_std'] = np.std(horizontal_proj)
    features['vertical_max'] = np.max(vertical_proj)
    features['vertical_std'] = np.std(vertical_proj)
    
    # 2. 密度分布
    threshold = 0.5
    density_above = np.sum(image > threshold) / (h * w)
    features['density_above_threshold'] = density_above
    
    # 3. Zernikeモーメント（近似）
    # 中心からの座標
    y, x = np.mgrid[0:h, 0:w]
    center_y, center_x = h//2, w//2
    
    # 中心からの座標（正規化）
    norm_y = (y - center_y) / (h/2)
    norm_x = (x - center_x) / (w/2)
    
    # 0次Zernikeモーメント（画像の重み合計）
    features['zernike_00'] = np.sum(image)
    
    # 1次モーメント（重心の近似）
    if features['zernike_00'] > 0:
        features['zernike_10'] = np.sum(norm_x * image) / features['zernike_00']
        features['zernike_01'] = np.sum(norm_y * image) / features['zernike_00']
    else:
        features['zernike_10'] = 0
        features['zernike_01'] = 0
    
    # 2次モーメント（慣性モーメントの近似）
    if features['zernike_00'] > 0:
        features['zernike_20'] = np.sum(norm_x**2 * image) / features['zernike_00']
        features['zernike_02'] = np.sum(norm_y**2 * image) / features['zernike_00']
        features['zernike_11'] = np.sum(norm_x * norm_y * image) / features['zernike_00']
    else:
        features['zernike_20'] = 0
        features['zernike_02'] = 0
        features['zernike_11'] = 0
    
    # 4. ラプラシアンエネルギー（エッジの鋭さ）
    laplacian = cv2.Laplacian(image.astype(np.float32), cv2.CV_64F)
    features['laplacian_energy'] = np.sum(np.abs(laplacian))
    
    # 5. HOG特徴（Histogram of Oriented Gradients）の簡易版
    # Sobelで勾配を計算
    sobel_x = cv2.Sobel(image.astype(np.float32), cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image.astype(np.float32), cv2.CV_64F, 0, 1, ksize=3)
    
    # 勾配の大きさと角度
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    angle = np.arctan2(sobel_y, sobel_x) * 180 / np.pi
    
    # 角度を9つのビンに分類
    angle_bins = np.digitize(angle, bins=np.linspace(-180, 180, 10)) - 1
    
    # HOGヒストグラム
    hog_hist = np.zeros(9)
    for i in range(9):
        hog_hist[i] = np.sum(magnitude[angle_bins == i])
    
    features['hog_max'] = np.max(hog_hist)
    features['hog_mean'] = np.mean(hog_hist)
    features['hog_std'] = np.std(hog_hist)
    
    return features

# 高度な特徴抽出の実例
sample_digits = [0, 3, 7]  # 代表的な数字を選択

for digit in sample_digits:
    print(f"\n=== Digit {digit} ===")
    
    # 画像生成と前処理
    original = create_handwritten_digit(digit)
    processed = preprocess_image(original)
    
    # 特徴抽出
    basic_features = extract_features(processed)
    advanced_features = extract_advanced_features(processed)
    
    print("基本特徴量:")
    for key, value in basic_features.items():
        if key != 'histogram':
            print(f"  {key}: {value:.4f}")
    
    print("\n高度な特徴量:")
    for key, value in advanced_features.items():
        if isinstance(value, np.ndarray) and value.ndim == 1:
            print(f"  {key}: mean={value.mean():.4f}, std={value.std():.4f}")
        else:
            print(f"  {key}: {value:.4f}")

## Exercise 2.4: 特徴の分布分析

異なる数字の特徴の分布を分析し、識別しやすい特徴を見つけよ。

In [ ]:
def analyze_feature_distribution(dataset, feature_name):
    """特定の特徴の分布を分析する"""
    feature_values = {i: [] for i in range(10)}
    
    # 各数字から特徴を抽出
    for digit in range(10):
        digit_indices = np.where(dataset['y_train'] == digit)[0]
        
        # サンプリング（計算時間節約）
        sample_indices = np.random.choice(digit_indices, min(50, len(digit_indices)), replace=False)
        
        for idx in sample_indices:
            features = extract_features(dataset['X_train'][idx])
            feature_values[digit].append(features[feature_name])
    
    # 分布を可視化
    plt.figure(figsize=(12, 6))
    
    # ヒストグラム
    plt.subplot(1, 2, 1)
    for digit in range(10):
        plt.hist(feature_values[digit], alpha=0.5, label=f'Digit {digit}', bins=20)
    plt.xlabel(feature_name)
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {feature_name}')
    plt.legend()
    
    # 箱ひげ図
    plt.subplot(1, 2, 2)
    plt.boxplot([feature_values[d] for d in range(10)], labels=[str(d) for d in range(10)])
    plt.xlabel('Digit')
    plt.ylabel(feature_name)
    plt.title(f'{feature_name} by Digit')
    
    plt.tight_layout()
    plt.show()
    
    return feature_values

# 有望な特徴のリスト
promising_features = [
    'mean',          # 平均輝度
    'std',           # 標準偏差
    'edge_magnitude', # エッジ強度
    'centroid_x',    # 重心X座標
    'variance_x',    # X方向の分散
    'aspect_ratio'   # アスペクト比
]

# 特徴の分布分析
for feature in promising_features:
    print(f"\n--- Analyzing {feature} ---")
    feature_values = analyze_feature_distribution(dataset, feature)
    
    # クラス間分離度を計算（Fisher's Discriminant Ratio）
    between_class_variance = 0
    within_class_variance = 0
    overall_mean = np.mean([np.mean(feature_values[d]) for d in range(10)])
    
    for digit in range(10):
        digit_mean = np.mean(feature_values[digit])
        digit_var = np.var(feature_values[digit])
        
        between_class_variance += len(feature_values[digit]) * (digit_mean - overall_mean)**2
        within_class_variance += (len(feature_values[digit]) - 1) * digit_var
    
    if within_class_variance > 0:
        fisher_ratio = between_class_variance / within_class_variance
        print(f"\nFisher's Discriminant Ratio: {fisher_ratio:.4f}")
        
        # Fisher比が大きいほどクラスが分離しやすい
        if fisher_ratio > 2:
            print(f"  ✅ {feature} はクラスを識別しやすい特徴量です")
        elif fisher_ratio > 1:
            print(f"  ⚠️  {feature} は中程度の識別力があります")
        else:
            print(f"  ❌ {feature} は識別にあまり役立ちません")

## Exercise 2.5: データセットの保存と読み込み

作成したデータセットをNumPy形式で保存し、後で読み込めるようにせよ。

In [ ]:
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def save_dataset_with_features(dataset, base_filename='digits_dataset'):
    """データセットと特徴を保存する"""
    print("データセットの準備と保存を開始...")
    
    # ファイルパス
    npy_dir = 'datasets'
    os.makedirs(npy_dir, exist_ok=True)
    
    # データセットを保存
    np.save(f'{npy_dir}/{base_filename}_X_train.npy', dataset['X_train'])
    np.save(f'{npy_dir}/{base_filename}_y_train.npy', dataset['y_train'])
    np.save(f'{npy_dir}/{base_filename}_X_test.npy', dataset['X_test'])
    np.save(f'{npy_dir}/{base_filename}_y_test.npy', dataset['y_test'])
    
    print(f"NumPy形式で保存完了: {npy_dir}/{base_filename}_*.npy")
    
    # 特徴データセットを作成
    print("特徴データセットを作成中...")
    
    # 訓練データから特徴を抽出（サンプリングして計算時間を節約）
    sample_size = min(1000, len(dataset['X_train']))
    sample_indices = np.random.choice(len(dataset['X_train']), sample_size, replace=False)
    
    X_features = []
    y_features = []
    
    for idx in sample_indices:
        features = extract_features(dataset['X_train'][idx])
        feature_vector = [
            features['mean'],
            features['std'],
            features['aspect_ratio'],
            features['centroid_x'] / 28,  # 正規化
            features['centroid_y'] / 28,  # 正規化
            features['edge_magnitude'],
            features['variance_x'] / (28**2),  # 正規化
            features['variance_y'] / (28**2)   # 正規化
        ]
        X_features.append(feature_vector)
        y_features.append(dataset['y_train'][idx])
    
    # 特徴データセットを保存
    np.save(f'{npy_dir}/{base_filename}_features.npy', np.array(X_features))
    np.save(f'{npy_dir}/{base_filename}_feature_labels.npy', np.array(y_features))
    
    print(f"特徴データセットを保存: {npy_dir}/{base_filename}_features.npy")
    
    # メタデータを保存
    metadata = {
        'num_train_samples': len(dataset['X_train']),
        'num_test_samples': len(dataset['X_test']),
        'num_features': 8,
        'image_shape': dataset['X_train'].shape[1:],
        'feature_names': [
            'mean', 'std', 'aspect_ratio', 
            'centroid_x', 'centroid_y', 
            'edge_magnitude', 'variance_x', 'variance_y'
        ]
    }
    
    with open(f'{npy_dir}/{base_filename}_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)
    
    print(f"メタデータを保存: {npy_dir}/{base_filename}_metadata.pkl")
    
    return npy_dir, base_filename

def load_dataset_with_features(npz_dir='datasets', base_filename='digits_dataset'):
    """保存されたデータセットを読み込む"""
    print("データセットの読み込みを開始...")
    
    # ディレクトリの確認
    if not os.path.exists(npz_dir):
        raise FileNotFoundError(f"ディレクトリ {npz_dir} が存在しません")
    
    # ファイルパス
    paths = {
        'X_train': f'{npz_dir}/{base_filename}_X_train.npy',
        'y_train': f'{npz_dir}/{base_filename}_y_train.npy',
        'X_test': f'{npz_dir}/{base_filename}_X_test.npy',
        'y_test': f'{npz_dir}/{base_filename}_y_test.npy',
        'features': f'{npz_dir}/{base_filename}_features.npy',
        'feature_labels': f'{npz_dir}/{base_filename}_feature_labels.npy',
        'metadata': f'{npz_dir}/{base_filename}_metadata.pkl'
    }
    
    # ファイルの存在確認
    for name, path in paths.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"ファイル {path} が存在しません")
    
    # データの読み込み
    loaded_data = {}
    for name in ['X_train', 'y_train', 'X_test', 'y_test', 'features', 'feature_labels']:
        loaded_data[name] = np.load(paths[name])
    
    # メタデータの読み込み
    with open(paths['metadata'], 'rb') as f:
        loaded_data['metadata'] = pickle.load(f)
    
    print("読み込んだデータセットの情報:")
    print(f"訓練画像: {loaded_data['X_train'].shape}")
    print(f"訓練ラベル: {loaded_data['y_train'].shape}")
    print(f"テスト画像: {loaded_data['X_test'].shape}")
    print(f"テストラベル: {loaded_data['y_test'].shape}")
    print(f"特徴ベクトル: {loaded_data['features'].shape}")
    print(f"特徴ラベル: {loaded_data['feature_labels'].shape}")
    
    return loaded_data

# 保存を実行（データセットが存在すれば）
if 'dataset' in locals() and len(dataset['X_train']) > 0:
    npz_dir, base_filename = save_dataset_with_features(dataset)
    
    # 読み込みのテスト
    loaded_data = load_dataset_with_features(npz_dir, base_filename)
    
    # データのサンプル表示
    plt.figure(figsize=(15, 5))
    
    # 元の画像のサンプル
    plt.subplot(1, 3, 1)
    sample_idx = 0
    plt.imshow(loaded_data['X_train'][sample_idx], cmap='gray')
    plt.title(f"Original Image (Label: {loaded_data['y_train'][sample_idx]})")
    plt.axis('off')
    
    # 特徴ベクトルの表示
    plt.subplot(1, 3, 2)
    feature_names = loaded_data['metadata']['feature_names']
    plt.barh(range(len(feature_names)), loaded_data['features'][sample_idx])
    plt.yticks(range(len(feature_names)), feature_names)
    plt.title("Feature Vector")
    plt.xlabel("Value")
    
    # 特徴の分布
    plt.subplot(1, 3, 3)
    for digit in range(10):
        digit_features = loaded_data['features'][loaded_data['feature_labels'] == digit]
        plt.scatter([digit] * len(digit_features), feature_names, 
                   alpha=0.3, label=f'Digit {digit}')
    
    plt.title("Feature Distribution by Digit")
    plt.xlabel("Digit")
    plt.ylabel("Feature Names")
    
    plt.tight_layout()
    plt.show()

## Exercise 2.6: データセットのバリデーション

保存したデータセットの整合性を確認し、問題がないことを検証せよ。

In [ ]:
def validate_dataset(dataset_path='datasets', dataset_name='digits_dataset'):
    """データセットの整合性を検証する"""
    print(f"データセット {dataset_name} の整合性検証...")
    
    # ファイルの存在確認
    required_files = [
        f'{dataset_path}/{dataset_name}_X_train.npy',
        f'{dataset_path}/{dataset_name}_y_train.npy',
        f'{dataset_path}/{dataset_name}_X_test.npy',
        f'{dataset_path}/{dataset_name}_y_test.npy',
        f'{dataset_path}/{dataset_name}_features.npy',
        f'{dataset_path}/{dataset_name}_metadata.pkl'
    ]
    
    print("\n1. ファイル存在チェック")
    for file in required_files:
        if os.path.exists(file):
            size = os.path.getsize(file) / (1024 * 1024)  # MB
            print(f"  ✅ {os.path.basename(file)} ({size:.2f} MB)")
        else:
            print(f"  ❌ {os.path.basename(file)} - ファイルが見つかりません")
            return False
    
    # データの読み込み
    print("\n2. データ読み込み")
    try:
        X_train = np.load(f'{dataset_path}/{dataset_name}_X_train.npy')
        y_train = np.load(f'{dataset_path}/{dataset_name}_y_train.npy')
        X_test = np.load(f'{dataset_path}/{dataset_name}_X_test.npy')
        y_test = np.load(f'{dataset_path}/{dataset_name}_y_test.npy')
        features = np.load(f'{dataset_path}/{dataset_name}_features.npy')
        
        with open(f'{dataset_path}/{dataset_name}_metadata.pkl', 'rb') as f:
            metadata = pickle.load(f)
        
        print("  ✅ すべてのファイルを読み込み成功")
        
    except Exception as e:
        print(f"  ❌ データ読み込みエラー: {e}")
        return False
    
    # データ形状の検証
    print("\n3. データ形状の検証")
    
    # ラベルの範囲チェック
    print(f"  - 訓練ラベルの範囲: {min(y_train)} - {max(y_train)}")
    print(f"  - テストラベルの範囲: {min(y_test)} - {max(y_test)}")
    
    # 画像形状のチェック
    print(f"  - 訓練画像の形状: {X_train.shape}")
    print(f"  - テスト画像の形状: {X_test.shape}")
    
    # 画像値の範囲チェック
    X_train_min, X_train_max = X_train.min(), X_train.max()
    X_test_min, X_test_max = X_test.min(), X_test.max()
    print(f"  - 訓練画像の値の範囲: [{X_train_min:.4f}, {X_train_max:.4f}]")
    print(f"  - テスト画像の値の範囲: [{X_test_min:.4f}, {X_test_max:.4f}]")
    
    # メタデータとの一致確認
    print("\n4. メタデータの検証")
    print(f"  - 訓練サンプル数: {metadata['num_train_samples']} (実数: {len(X_train)})")
    print(f"  - テストサンプル数: {metadata['num_test_samples']} (実数: {len(X_test)})")
    print(f"  - 画像形状: {metadata['image_shape']} (実数: {X_train.shape[1:]})")
    
    # データのランダムサンプリングで表示
    print("\n5. データサンプルの確認")
    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    axes = axes.flatten()
    
    for i, ax in enumerate(axes):
        # ランダムにインデックスを選択
        idx = np.random.randint(len(X_train))
        ax.imshow(X_train[idx], cmap='gray')
        ax.set_title(f"Label: {y_train[idx]}")
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # データ分布の検証
    print("\n6. データ分布の検証")
    
    plt.figure(figsize=(12, 4))
    
    # 訓練データの分布
    plt.subplot(1, 2, 1)
    unique, counts = np.unique(y_train, return_counts=True)
    plt.bar(unique, counts)
    plt.title('Training Data Distribution')
    plt.xlabel('Digit')
    plt.ylabel('Count')
    
    # テストデータの分布
    plt.subplot(1, 2, 2)
    unique, counts = np.unique(y_test, return_counts=True)
    plt.bar(unique, counts)
    plt.title('Test Data Distribution')
    plt.xlabel('Digit')
    plt.ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ データセットの整合性検証が完了しました")
    return True

# データセットの検証（保存されている場合）
if os.path.exists('datasets'):
    validation_result = validate_dataset()
    if validation_result:
        print("\nデータセットは正常に使用できます")
    else:
        print("\nデータセットに問題が検出されました")
else:
    print("保存されたデータセットが見つかりません")

## Challenge: 高度なデータ拡張

GAN（敵対的生成ネットワーク）を用いて、より現実的な手書き数字を生成するプロトタイプを実装せよ。
（ヒント: 事前学習モデルを利用またはシンプルなGANを実装）

In [ ]:
# Challenge問題: GANによる手書き数字生成
# 時間の関係で、シンプルなGANの実装を示します

class SimpleGAN:
    """シンプルなGANの実装（概念）"""
    def __init__(self, latent_dim=100, image_shape=(28, 28)):
        self.latent_dim = latent_dim
        self.image_shape = image_shape
        self.generator = None
        self.discriminator = None
        
    def build_generator(self):
        """生成器を構築（実際の実装ではKeras/TensorFlow/PyTorchを使用）"""
        # ここではダミーの実装
        print("Generator: ノイズから画像を生成するニューラルネットワーク")
        print(f"  入力: 潜在空間のベクトル (size={self.latent_dim})")
        print(f"  出力: {self.image_shape} の画像")
        return "Generator built"
    
    def build_discriminator(self):
        """識別器を構築"""
        # ここではダミーの実装
        print("Discriminator: 本物/偽物を識別するニューラルネットワーク")
        print(f"  入力: {self.image_shape} の画像")
        print("  出力: 本物確率（0-1）")
        return "Discriminator built"
    
    def train(self, dataset, epochs=10):
        """GANを訓練"""
        print(f"GANの訓練を開始... (Epochs: {epochs})")
        print("実際の実装では、KerasやPyTorchを使用")
        
        # 訓練ループの概念
        for epoch in range(epochs):
            # 1. 識別器の訓練
            # 2. 生成器の訓練
            # 3. 損失の計算
            print(f"  Epoch {epoch+1}/{epochs} - completed")
        
        print("訓練完了")
        return self
    
    def generate_images(self, num_images=10):
        """画像を生成"""
        print(f"{num_images} 枚の画像を生成...")
        print("実際には、訓練済みの生成器を使用")
        
        # ダミーの生成（ランダムノイズ）
        dummy_images = []
        for i in range(num_images):
            # 乱数ベクトルから画像を生成
            noise = np.random.randn(self.latent_dim)
            # 生成器による変換（ここでは単なる乱数の可視化）
            fake_image = np.random.rand(*self.image_shape) * 0.5 + 0.25
            dummy_images.append(fake_image)
        
        return dummy_images

# GANの使用例
print("\n=== GANによる手書き数字生成の概念 ===")
gan = SimpleGAN()
gan.build_generator()
gan.build_discriminator()

# 訓練（実際には多くのデータと時間が必要）
# gan.train(dataset, epochs=100)

# 生成
generated_images = gan.generate_images(10)

# 生成された画像の表示
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < len(generated_images):
        ax.imshow(generated_images[i], cmap='gray')
        ax.set_title(f'Generated {i}')
        ax.axis('off')
    else:
        ax.axis('off')

plt.suptitle('GAN Generated Images (Conceptual)', fontsize=16)
plt.tight_layout()
plt.show()

print("\n実際のGANの実装には以下の技術が必要です:")
print("- ディープラーニングフレームワーク (TensorFlow/Keras, PyTorch)")
print("- バッチ正規化とドロップアウト")
print("- オプティマイザ（Adamなど）")
print("- 損失関数（二値交差エントロピー）")
print("- 勾配クリッピングなど）")

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] 手書き数字認識の目的とMNISTデータセットの特徴を理解した
- [ ] 画像前処理パイプラインの各ステップを説明できる
- [ ] 特徴抽出の目的と重要性を理解した
- [ ] データ拡張の意義と手法を理解した

## 技術要素
- [ ] 画像生成アルゴリズムを理解した
- [ ] 前処理手法（正規化、二値化、ノイズ除去など）を実装できる
- [ ] 基本的な特徴量（統計量、重心、エッジなど）を抽出できる
- [ ] データ拡張の様々な手法を実装できる

## 実践力
- [ ] 手書き数字データセットを生成した
- [ ] データの保存と読み込みを実装した
- [ ] 特徴分布分析を実行した
- [ ] データセットの整合性検証を行った

## 発展的トピック
- [ ] GANの基本概念を理解した
- [ ] 高度な特徴量（Zernikeモーメント、HOGなど）を理解した
- [ ] データ拡張の戦略（回転、スケーリング、明度変更）を設計した

---

**お疲れ様でした！** Day 22はこれで終了です。

次回（Day 23）は「分類器の実装」を学び、k-NNなどのアルゴリズムを実装します。

復習課題：
1. 生成したデータセットをロードし、特徴量を可視化する
2. 異なる前処理手法の効果を比較する
3. データ拡張が認識性能に与える影響を分析する